## Download the data

In [1]:
import os.path
if not os.path.isfile('data/vmware_ir_content.csv'):
    !pip install kaggle
    !kaggle competitions download -c vmware-zero-shot-information-retrieval
    !mkdir -p data/
    !unzip -o vmware-zero-shot-information-retrieval.zip
    !mv *.csv data/

## Parse the vmware corpus

- Load vmware text
- Read metadata into fields

In [15]:
import pandas as pd
corpus = pd.read_csv('data/vmware_ir_content.csv')
corpus = corpus.fillna('')

import json
import ast

parsed_rows = []

for row in corpus.to_dict(orient='records'):
    row_dict = ast.literal_eval(row['raw_meta'])
    row_dict['id'] = row['f_name']
    row_dict['f_name'] = row['f_name']
    row_dict['raw_text'] = row['raw_text']
    row_dict['document_group'] = row['document_group']

    # fields that dont index for some reason
    bad_fields = ["DC.Date", "DC.Publisher"]
    for bad_field in bad_fields:
        if bad_field in row_dict:
            del row_dict[bad_field]
        
    parsed_rows.append(row_dict)
    
corpus = pd.DataFrame(parsed_rows)
corpus

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
0,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Black Duck for VMware Tanzu | Solutions Hub | ...,authenticity_token,NnLorKoXWVAXtOGcZgn6cD0qO7KbHyuGc6QwI0FaGybXtO...,Black Duck for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Percona | Solutions Hub | VMware Tanzu,authenticity_token,2N9nj_cHWOjsFUJe2rbR8Rlllj9DfNaD9m79W4GYB9IlnP...,Percona | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,United States Air Force | Customers | VMware ...,authenticity_token,CCmw9wy9YaWEur35yHQYIvF6XbSJ23oUmZsF2qf5Qmyo83...,United States Air Force | Customers,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Dynatrace for VMware Tanzu | Solutions Hub | V...,authenticity_token,VqAn3GNy9pSHWoDrlxi_mxbAM6XrBFTcIDQeYUxvWCehtX...,Dynatrace for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,New Relic for VMware Tanzu | Solutions Hub | V...,authenticity_token,VM0vueZ5tE_2BnTwD-f09O5A3bKGe-eCZO2ixNeHAzd-hW...,New Relic for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from ltr.client import ElasticClient
client=ElasticClient()

from ltr.index import rebuild
rebuild(client, index='vmware', doc_src=parsed_rows, force=True)

Deleted index vmware [Status: 200]
Created index vmware [Status: 200]
